In [ ]:
import geopandas as gpd
import pandas as pd

# Read in data
df = gpd.read_parquet("feds_western_us_2013_af_postprocessed.parquet")
print("COLUMNS:", df.columns)
print("CRS:", df.crs)
print(df.head())


COLUMNS: Index(['mergeid', 'invalid', 'ftype', 'n_pixels', 'n_newpixels', 'farea',
       'fperim', 'flinelen', 'duration', 'pixden', 'meanFRP', 't_st', 't_ed',
       'hull', 'fline', 'nfp'],
      dtype='object')
CRS: {"$schema": "https://proj.org/schemas/v0.6/projjson.schema.json", "type": "ProjectedCRS", "name": "NAD27 / US National Atlas Equal Area", "base_crs": {"name": "NAD27", "datum": {"type": "GeodeticReferenceFrame", "name": "North American Datum 1927", "ellipsoid": {"name": "Clarke 1866", "semi_major_axis": 6378206.4, "semi_minor_axis": 6356583.8}}, "coordinate_system": {"subtype": "ellipsoidal", "axis": [{"name": "Geodetic latitude", "abbreviation": "Lat", "direction": "north", "unit": "degree"}, {"name": "Geodetic longitude", "abbreviation": "Lon", "direction": "east", "unit": "degree"}]}, "id": {"authority": "EPSG", "code": 4267}}, "conversion": {"name": "US National Atlas Equal Area", "method": {"name": "Lambert Azimuthal Equal Area (Spherical)", "id": {"authority": "EP

In [ ]:
#Find the longest duration fire and subset it for initial pipeline development
df_flat = df.reset_index()
max_area = df_flat.farea.idxmax()
max_fire_id = df_flat.loc[max_area,"fireID"]
hero_fire = df_flat.loc[df_flat.fireID == max_fire_id,:]
hero_fire.head(5)


,t,fireID,mergeid,invalid,ftype,n_pixels,n_newpixels,farea,fperim,flinelen,duration,pixden,meanFRP,t_st,t_ed,hull,fline,nfp
54703,2013-08-20 00:00:00,60839.0,60839.0,False,4,192,122,36.992059,42.787746,28.949071,2.0,5.190303,8.876230,2013-08-18,2013-08-20 00:00:00,"POLYGON ((-1748148.199 -582120.982, -1748516.6...","MULTILINESTRING ((-1748232.971 -581942.536, -1...","MULTIPOINT ((-1747743.257 -581060.276), (-1747..."
54744,2013-08-22 12:00:00,60839.0,60839.0,False,4,2442,729,332.840034,98.624595,70.752038,4.5,7.336858,163.508573,2013-08-18,2013-08-22 12:00:00,"POLYGON ((-1750822.016 -579886.26, -1750823.17...","MULTILINESTRING ((-1750819.051 -579904.497, -1...","MULTIPOINT ((-1736702.265 -590130.259), (-1737..."
54764,2013-10-30 12:00:00,60839.0,60839.0,False,2,16942,0,1142.871586,177.376102,0.000000,69.0,14.824063,NaN,2013-08-18,2013-10-26 00:00:00,"POLYGON ((-1753426.387 -577125.971, -1753437.5...",None,None
54765,2013-10-17 00:00:00,60839.0,60839.0,False,4,16933,0,1142.667589,177.237862,0.000000,49.0,14.818833,NaN,2013-08-18,2013-10-06 00:00:00,"POLYGON ((-1753426.387 -577125.971, -1753437.5...",None,None
54766,2013-11-01 00:00:00,60839.0,60839.0,False,2,16944,0,1144.137238,177.578393,1.715400,74.5,14.809412,NaN,2013-08-18,2013-10-31 12:00:00,"POLYGON ((-1753426.387 -577125.971, -1753437.5...","LINESTRING (-1740314.682 -560217.488, -1739909...",None


In [3]:
from geocube.api.core import make_geocube
hero_fire = hero_fire.rename_geometry("geometry")

# 2. Now run the exact same code as before
out_grid = make_geocube(
    vector_data=hero_fire,
    resolution=(-500, 500),
    measurements=["fireID"],
    output_crs="EPSG:9311",
    group_by="t"
)

print(out_grid)

<xarray.Dataset> Size: 11MB
Dimensions:      (t: 159, y: 83, x: 102)
Coordinates:
  * t            (t) datetime64[ns] 1kB 2013-08-18 ... 2013-11-05
  * y            (y) float64 664B -5.582e+05 -5.588e+05 ... -5.992e+05
  * x            (x) float64 816B -1.759e+06 -1.758e+06 ... -1.708e+06
    spatial_ref  int64 8B 0
Data variables:
    fireID       (t, y, x) float64 11MB nan nan nan nan nan ... nan nan nan nan


## Okay, now I've got a simple grid for a single fire.. time to get weather data for it.

In [ ]:
# 1. Get the tight bounds of the fire (minx, miny, maxx, maxy)
minx, miny, maxx, maxy = hero_fire.total_bounds

# 2. Define a buffer (e.g., 10km = 10,000 meters)
BUFFER = 10000

# 3. Create the "Weather Box"
weather_box = [
    minx - BUFFER,  # West edge moves further West
    miny - BUFFER,  # South edge moves further South
    maxx + BUFFER,  # East edge moves further East
    maxy + BUFFER   # North edge moves further North
]

print(f"Fire Bounds:    {hero_fire.total_bounds}")
print(f"Weather Bounds: {weather_box}")

# 4. Define the Time Window
start_time = hero_fire.t.min()
end_time = hero_fire.t.max()

print(f"Need Weather from: {start_time} to {end_time}")